In [1]:
import pandas as pd
import numpy as np
import math
from geopy import distance

In [2]:
speeds = pd.read_csv("raw/PHH_Speeds_Current-PHH_Vitesses_Actuelles_BC.csv")

# Same source, map data with population information for each hex
phh = pd.read_csv("raw/PHH-BC.csv")
phhWithSpeeds = pd.merge(phh, speeds, on="PHH_ID")

# Filtering out hexes with no population
regionsWithPop = phhWithSpeeds[phhWithSpeeds["Pop2016"] > 0].copy()
regionsWithPop.columns

Index(['PHH_ID', 'Type', 'Pop2016', 'TDwell2016_TLog2016',
       'URDwell2016_RH2016', 'DBUID_Ididu', 'HEXUID_IdUHEX', 'Pruid_Pridu',
       'Latitude', 'Longitude', 'Combined_lt5_1_Combine',
       'Wired_lt5_1_Filaire', 'Wireless_lt5_Sans_fil', 'Combined_5_1_Combine',
       'Wired_5_1_Filaire', 'Wireless_5_1_Sans_fil', 'Combined_10_2_Combine',
       'Wired_10_2_Filaire', 'Wireless_10_2_Sans_fil', 'Combined_25_5_Combine',
       'Wired_25_5_Filaire', 'Wireless_25_5_Sans_fil',
       'Combined_50_10_Combine', 'Wired_50_10_Filaire',
       'Wireless_50_10_Sans_fil', 'Combined_Max_Threshold-Combine_Seuil_Max',
       'Wired_Max_Threshold-Filaire_Seuil_Max',
       'Wireless_Max_Threshold-Sans_fil_Seuil_Max', 'Avail_LTE_Mobile_Dispo'],
      dtype='object')

In [18]:
exam_info = pd.read_csv("raw/provincial-required-examinations-2016_2017_to_2019_2020.csv")
exam_info = exam_info[exam_info["SCHOOL_YEAR"] == "2018/2019"]
exam_info = exam_info[exam_info["DATA_LEVEL"] == "School Level"]
exam_info = exam_info[exam_info["EXAM_SUBJECT"] == "ENGLISH 12"]
exam_info = exam_info[exam_info["SUB_POPULATION"] == "All Students"]
exam_info = exam_info[exam_info["MARK_TYPE"] == "Exam Marks"]
per_school = exam_info.groupby("SCHOOL_NUMBER").first()
unmasked = per_school[per_school["AVERAGE_PERCENT"] != "Msk"]

In [21]:
schools = pd.read_csv("raw/allschools_databc_20201027-fewer-columns.csv")
schools = schools[["SCHOOL_NUMBER", "SCHOOL_LATITUDE", "SCHOOL_LONGITUDE"]]
unmasked_with_location = unmasked.merge(schools, left_on="SCHOOL_NUMBER", right_on="SCHOOL_NUMBER")
unmasked_with_location

,SCHOOL_NUMBER,SCHOOL_YEAR,DATA_LEVEL,PUBLIC_OR_INDEPENDENT,DISTRICT_NUMBER,DISTRICT_NAME,SCHOOL_NAME,FACILITY_TYPE,SUBJECT_CATEGORY,EXAM_SUBJECT,...,NUMBER_OF_C_PLUS,PERCENT_OF_C_PLUS,NUMBER_OF_C,PERCENT_OF_C,NUMBER_OF_C_MINUS,PERCENT_OF_C_MINUS,NUMBER_OF_F,PERCENT_OF_F,SCHOOL_LATITUDE,SCHOOL_LONGITUDE
0,502001.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Mount Baker Secondary,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,51,26,46,23,20,10,Msk,Msk,49.512677,-115.760843
1,505033.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Elkford Elementary Secondary,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,50.006840,-114.930648
2,505034.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Fernie Secondary,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,11,26,Msk,Msk,Msk,Msk,Msk,Msk,49.513593,-115.053144
3,505035.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Sparwood Secondary,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,49.735726,-114.891552
4,596715.0,2018/2019,School Level,BC Independent School,NaN,The Fernie Academy Society (IATHNBR:509),The Fernie Academy,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,49.502830,-115.062157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,9357012.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Secondaire Duchess-Park,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,53.916494,-122.758214
344,9361060.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Victor-Brodeur,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,48.431831,-123.401138
345,9368001.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Secondaire de Nanaimo,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,49.161041,-123.962077
346,9393002.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Gabrielle-Roy,All Facility Types,LANGUAGE ARTS,ENGLISH 12,...,10,25,Msk,Msk,Msk,Msk,Msk,Msk,49.128397,-122.858321


In [22]:
regionsWithPop["DISTANCE"] = -1
regionsWithPop["SCHOOL_NUMBER"] = -1
for index, hexagon in regionsWithPop.iterrows():
    hexagon_coord = (hexagon["Latitude"], hexagon["Longitude"])
    closest_school_num = None
    closest_distance = None
    for i, school in unmasked_with_location.iterrows():
        dist = distance.distance(hexagon_coord, (school["SCHOOL_LATITUDE"], school["SCHOOL_LONGITUDE"])).km
        if (closest_distance is None or closest_distance > dist):
            closest_distance = dist
            closest_school_num = school["SCHOOL_NUMBER"]
    regionsWithPop.loc[index, "DISTANCE"] = closest_distance
    regionsWithPop.loc[index, "SCHOOL_NUMBER"] = closest_school_num

regionsWithPop.to_csv("pop_regions_with_school_secondary.csv")
regionsWithPop.groupby("SCHOOL_NUMBER").size().reset_index(name='counts')

,SCHOOL_NUMBER,counts
0,502001,4295
1,505033,352
2,505034,521
3,505035,659
4,596715,1660
...,...,...
333,9199085,277
334,9343092,790
335,9361060,870
336,9393002,362


In [24]:
temp = pd.read_csv("raw/provincial-required-examinations-2016_2017_to_2019_2020.csv")
temp = temp[temp["SCHOOL_YEAR"] == "2018/2019"]
temp = temp[temp["DATA_LEVEL"] == "School Level"]
temp.groupby("SCHOOL_NUMBER").first()

,SCHOOL_YEAR,DATA_LEVEL,PUBLIC_OR_INDEPENDENT,DISTRICT_NUMBER,DISTRICT_NAME,SCHOOL_NAME,FACILITY_TYPE,SUBJECT_CATEGORY,EXAM_SUBJECT,COURSE_CODE,...,NUMBER_OF_B,PERCENT_OF_B,NUMBER_OF_C_PLUS,PERCENT_OF_C_PLUS,NUMBER_OF_C,PERCENT_OF_C,NUMBER_OF_C_MINUS,PERCENT_OF_C_MINUS,NUMBER_OF_F,PERCENT_OF_F
SCHOOL_NUMBER,,,,,,,,,,,,,,,,,,,,,
502001.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Mount Baker Secondary,All Facility Types,ENGLISH,ENGLISH 12 FIRST PEOPLES,EFP_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
505000.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Kootenay Learning Campus (KLC),All Facility Types,LANGUAGE ARTS,COMMUNICATIONS 12,COM_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
505033.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Elkford Elementary Secondary,All Facility Types,LANGUAGE ARTS,COMMUNICATIONS 12,COM_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
505034.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Fernie Secondary,All Facility Types,LANGUAGE ARTS,COMMUNICATIONS 12,COM_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
505035.0,2018/2019,School Level,BC Public School,5.0,Southeast Kootenay,Sparwood Secondary,All Facility Types,LANGUAGE ARTS,COMMUNICATIONS 12,COM_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9393002.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Gabrielle-Roy,All Facility Types,LANGUAGE ARTS,ENGLISH 12,EN_12_C,...,18,45,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
9393004.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Secondaire de Penticton,All Facility Types,LANGUAGE ARTS,ENGLISH 12,EN_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
9393005.0,2018/2019,School Level,BC Public School,93.0,Conseil scolaire francophone,Secondaire Jules-Verne,All Facility Types,LANGUAGE ARTS,COMMUNICATIONS 12,COM_12_C,...,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk,Msk
